In [1]:
#default_exp reader.alphapept_reader

In [2]:
#export
import numba
import os
import pandas as pd
import h5py

from alphadeep.reader.psm_reader import PSMReaderBase

@numba.njit
def parse_ap(precursor):
    """
    Parser to parse peptide strings
    """
    items = precursor.split('_')
    if len(items) == 3:
        decoy = 1
    else:
        decoy = 0
    modseq = items[0]
    charge = items[-1]

    parsed = []
    mods = []
    sites = []
    string = ""

    if modseq[0] == 'a':
        sites.append('0')
        mods.append('a')
        modseq = modseq[1:]

    for i in modseq:
        string += i
        if i.isupper():
            parsed.append(i)
            if len(string) > 1:
                sites.append(str(len(parsed)))
                mods.append(string)
            string = ""

    return ''.join(parsed), ';'.join(mods), ';'.join(sites), charge, decoy

class AlphaPeptReader(PSMReaderBase):
    def __init__(self, 
        frag_types=['b','y','b-modloss','y-modloss'], 
        max_frag_charge=2
    ):
        super().__init__(frag_types, max_frag_charge)

        self.modification_convert_dict['cC'] = 'Carbamidomethyl@C'
        self.modification_convert_dict['oxM'] = 'Oxidation@M'
        self.modification_convert_dict['pS'] = 'Phospho@S'
        self.modification_convert_dict['pT'] = 'Phospho@T'
        self.modification_convert_dict['pY'] = 'Phospho@Y'
        self.modification_convert_dict['a'] = 'Acetyl@Protein_N-term'

    def _load_file(self, filename):
        with h5py.File(filename, 'r') as _hdf:
            dataset = _hdf['peptide_fdr']
            df = pd.DataFrame({col:dataset[col] for col in dataset.keys()})

        psm_df = pd.DataFrame()

        psm_df['precursor'] = df['precursor'].str.decode('utf-8')

        psm_df['raw_name'] = os.path.basename(filename)[:-len('.ms_data.hdf')]
        psm_df['RT'] = df['rt']*60
        if 'scan_no' in df.columns:
            psm_df['scan'] = df['scan_no'].astype('int')
        else:
            psm_df['scan'] = pd.NA
        if 'mobility' in df.columns:
            psm_df['mobility'] = df['mobility']
        else:
            psm_df['mobility'] = pd.NA

        psm_df['score'] = df['score']
        
        psm_df['sequence'], psm_df['mods'], \
            psm_df['mod_sites'], psm_df['charge'], \
            psm_df['decoy'] = zip(*psm_df['precursor'].apply(parse_ap))
        psm_df['charge'] = psm_df['charge'].astype(int)
        psm_df['nAA'] = psm_df.sequence.str.len()

        self._psm_df = psm_df
    

In [3]:
#hide
filename = '/Users/zengwenfeng/Workspace/Data/PXD006109/20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgun_170519010518.ms_data.hdf'
ap_reader = AlphaPeptReader()
ap_reader.load(filename)
ap_reader.psm_df

,precursor,raw_name,RT,scan,mobility,score,sequence,mods,mod_sites,charge,decoy,nAA
0,AVVPSK_2,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,12.584717,7698,<NA>,0.696928,AVVPSK,,,2,0,6
1,ALGALR_2,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,22.885657,17422,<NA>,0.706138,ALGALR,,,2,0,6
2,LLAAGR_2,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,12.857236,7600,<NA>,0.724405,LLAAGR,,,2,0,6
3,VALVAK_2,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,22.165872,16148,<NA>,0.573965,VALVAK,,,2,0,6
4,VAIIGK_2,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,27.361039,20821,<NA>,0.632588,VAIIGK,,,2,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
38069,ENENGEEEEEEAEFGEEDLFHQQGDPR_3,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,61.068598,54356,<NA>,0.980562,ENENGEEEEEEAEFGEEDLFHQQGDPR,,,3,0,27
38070,NMITQYWPDRETAPGDISPYTIPEEDR_3,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,81.592126,73901,<NA>,0.947413,NMITQYWPDRETAPGDISPYTIPEEDR,,,3,0,27
38071,SEEoxMQTVQQEQLLQETQALQQSFLSEK_3,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,93.411330,85491,<NA>,0.991908,SEEMQTVQQEQLLQETQALQQSFLSEK,Oxidation@M,4,3,0,27
38072,VSILDENIAHDDKPGLYFHEEYVDMcCR_5,20170518_QEp1_FlMe_SA_BOX0_HeLa2_Ecoli1_Shotgu...,72.860629,65854,<NA>,0.954337,VSILDENIAHDDKPGLYFHEEYVDMCR,Carbamidomethyl@C,26,5,0,27
